In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
import os

In [2]:
cv=LeaveOneOut()

In [3]:
#X=pd.read_csv("Kmer4.csv",header=None)
y=pd.read_csv("label.csv",header=None)
y=np.array(y)
#print(X.shape)

In [4]:
def my_classifier1(X,y):
    print(datetime.datetime.now())
    TN_a=[]
    FN_a=[]
    FP_a=[]
    TP_a=[]
    accuracy_a=[]
    pre_a=[]
    rec_a=[]
    f1_a=[]
    auroc_a=[]
    param1=["best","random"]
    score=0
    best_param=param1[0]
    for i in param1:
        Predict_Y=[]    #存放预测值
        Prob_Y=[]       #存放预测概率
        Really_Y=y
        for train_index,test_index in cv.split(X):
            estimator=DecisionTreeClassifier(random_state=10,splitter=i)
            estimator.fit(X[train_index],y[train_index])
            Predict_Y.append(estimator.predict(X[test_index])[0])
            Prob_Y.append(estimator.predict_proba(X[test_index]))
        Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
        Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
        Prob_Y=pd.DataFrame(Prob_Y)
        Prob_Y.columns=["0","1"]
        confusion_matrix(Really_Y,Predict_Y)
        TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
        FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
        FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
        TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
        ACC=accuracy_score(Really_Y,Predict_Y)
        Precision=precision_score(Really_Y,Predict_Y)
        Recall=recall_score(Really_Y,Predict_Y)
        F1_score=f1_score(Really_Y,Predict_Y)
        AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
        if ACC>score:
            score=ACC
            best_param=i
        TN_a.append(TN)
        FN_a.append(FN)
        FP_a.append(FP)
        TP_a.append(TP)
        accuracy_a.append(ACC)
        pre_a.append(Precision)
        rec_a.append(Recall)
        f1_a.append(F1_score)
        auroc_a.append(AUC)  
    results=np.array([param1,TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a])
    result_final = pd.DataFrame(results)
    return best_param,results

In [5]:
def my_classifier2(p1,X,y):
    print(datetime.datetime.now())
    param2=np.array(range(3,10,1))
   # print(datetime.datetime.now())
    TN_a=[]
    FN_a=[]
    FP_a=[]
    TP_a=[]
    accuracy_a=[]
    pre_a=[]
    rec_a=[]
    f1_a=[]
    auroc_a=[]
    score=0
    best_param=param2[0]
    #param1=["best","random"]
    for i in param2:
        Predict_Y=[]    #存放预测值
        Prob_Y=[]       #存放预测概率
        Really_Y=y
        for train_index,test_index in cv.split(X):
            estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=i)
            estimator.fit(X[train_index],y[train_index])
            Predict_Y.append(estimator.predict(X[test_index])[0])
            Prob_Y.append(estimator.predict_proba(X[test_index]))
        Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
        Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
        Prob_Y=pd.DataFrame(Prob_Y)
        Prob_Y.columns=["0","1"]
        confusion_matrix(Really_Y,Predict_Y)
        TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
        FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
        FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
        TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
        ACC=accuracy_score(Really_Y,Predict_Y)
        if ACC>score:
            score=ACC
            best_param=i
        Precision=precision_score(Really_Y,Predict_Y)
        Recall=recall_score(Really_Y,Predict_Y)
        F1_score=f1_score(Really_Y,Predict_Y)
        AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
        TN_a.append(TN)
        FN_a.append(FN)
        FP_a.append(FP)
        TP_a.append(TP)
        accuracy_a.append(ACC)
        pre_a.append(Precision)
        rec_a.append(Recall)
        f1_a.append(F1_score)
        auroc_a.append(AUC)  
    results=np.array([param2,TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a])
    result_final = pd.DataFrame(results)
  #  return grid_search.best_params_["splitter"],results
    return best_param,results

In [6]:
def my_classifier(p1,p2,X, y):
    print(datetime.datetime.now())
   # param2=["best","random"]   ##设置splitter范围
   # param3=np.array(range(3,10,1))   ##设置max_depth范围
    i=X.shape[1]
    param3=np.array(range(10,i,100))  ## 设置max_features范围
  #  param2=np.array(range(3,10,1))
   # print(datetime.datetime.now())
    TN_a=[]
    FN_a=[]
    FP_a=[]
    TP_a=[]
    accuracy_a=[]
    pre_a=[]
    rec_a=[]
    f1_a=[]
    auroc_a=[]
    #param1=["best","random"]
    for j in param3:
        Predict_Y=[]    #存放预测值
        Prob_Y=[]       #存放预测概率
        Really_Y=y
        for train_index,test_index in cv.split(X):
            estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2,max_features=j)
            estimator.fit(X[train_index],y[train_index])
            Predict_Y.append(estimator.predict(X[test_index])[0])
            Prob_Y.append(estimator.predict_proba(X[test_index]))
        Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
        Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
        Prob_Y=pd.DataFrame(Prob_Y)
        Prob_Y.columns=["0","1"]
        confusion_matrix(Really_Y,Predict_Y)
        TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
        FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
        FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
        TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
        ACC=accuracy_score(Really_Y,Predict_Y)
        Precision=precision_score(Really_Y,Predict_Y)
        Recall=recall_score(Really_Y,Predict_Y)
        F1_score=f1_score(Really_Y,Predict_Y)
        AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
        TN_a.append(TN)
        FN_a.append(FN)
        FP_a.append(FP)
        TP_a.append(TP)
        accuracy_a.append(ACC)
        pre_a.append(Precision)
        rec_a.append(Recall)
        f1_a.append(F1_score)
        auroc_a.append(AUC)  
    results=np.array([param3,TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a])
    return results

In [8]:
file_path = 'E:\datatest\PseDPC\d10\d101'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
1011.csv
**************************************************
2022-02-24 22:44:21.418582
2022-02-24 22:44:28.060811
2022-02-24 22:45:00.946824
##################################################
1012.csv
**************************************************
2022-02-24 22:45:27.477862
2022-02-24 22:45:33.483797
2022-02-24 22:45:43.353393
##################################################
1013.csv
**************************************************
2022-02-24 22:45:51.265222
2022-02-24 22:45:57.564344
2022-02-24 22:46:29.450035
##################################################
1014.csv
**************************************************
2022-02-24 22:46:56.221408
2022-02-24 22:47:02.341035
2022-02-24 22:47:32.422552
##################################################
1015.csv
**************************************************
2022-02-24 22:47:54.030739
2022-02-24 22:48:00.065593
2022-02-24 22:48:30.272774
########################################

In [9]:
file_path = 'E:\datatest\PseDPC\d10\d102'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
1021.csv
**************************************************
2022-02-24 22:52:49.105269
2022-02-24 22:52:55.041414
2022-02-24 22:53:23.840336
##################################################
1022.csv
**************************************************
2022-02-24 22:53:50.164904
2022-02-24 22:53:56.189785
2022-02-24 22:54:25.607112
##################################################
1023.csv
**************************************************
2022-02-24 22:54:46.355567
2022-02-24 22:54:52.788355
2022-02-24 22:55:02.234084
##################################################
1024.csv
**************************************************
2022-02-24 22:55:12.786871
2022-02-24 22:55:19.311393
2022-02-24 22:55:52.096706
##################################################
1025.csv
**************************************************
2022-02-24 22:56:18.739395
2022-02-24 22:56:25.210082
2022-02-24 22:56:57.890645
########################################

In [10]:
file_path = 'E:\datatest\PseDPC\d10\d103'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
1031.csv
**************************************************
2022-02-24 23:01:08.315634
2022-02-24 23:01:13.967513
2022-02-24 23:01:40.684033
##################################################
1032.csv
**************************************************
2022-02-24 23:02:06.104022
2022-02-24 23:02:11.752938
2022-02-24 23:02:38.960116
##################################################
1033.csv
**************************************************
2022-02-24 23:02:59.606875
2022-02-24 23:03:05.298647
2022-02-24 23:03:33.026462
##################################################
1034.csv
**************************************************
2022-02-24 23:03:53.881663
2022-02-24 23:03:59.889589
2022-02-24 23:04:29.024660
##################################################
1035.csv
**************************************************
2022-02-24 23:04:49.624524
2022-02-24 23:04:55.743154
2022-02-24 23:05:26.368217
########################################

In [11]:
file_path = 'E:\datatest\PseDPC\d10\d104'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
1041.csv
**************************************************
2022-02-24 23:10:14.231054
2022-02-24 23:10:20.038516
2022-02-24 23:10:48.244080
##################################################
1042.csv
**************************************************
2022-02-24 23:11:14.724205
2022-02-24 23:11:20.957549
2022-02-24 23:11:52.008472
##################################################
1043.csv
**************************************************
2022-02-24 23:12:24.754347
2022-02-24 23:12:31.797931
2022-02-24 23:13:08.370498
##################################################
1044.csv
**************************************************
2022-02-24 23:13:41.176000
2022-02-24 23:13:48.900335
2022-02-24 23:14:26.375070
##################################################
1045.csv
**************************************************
2022-02-24 23:14:49.988914
2022-02-24 23:14:56.713899
2022-02-24 23:15:35.932191
########################################

In [12]:
file_path = 'E:\datatest\PseDPC\d10\d105'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
1051.csv
**************************************************
2022-02-24 23:20:35.175564
2022-02-24 23:20:41.295220
2022-02-24 23:21:12.547576
##################################################
1052.csv
**************************************************
2022-02-24 23:21:43.596539
2022-02-24 23:21:50.917535
2022-02-24 23:22:25.285584
##################################################
1053.csv
**************************************************
2022-02-24 23:22:55.979463
2022-02-24 23:23:05.998707
2022-02-24 23:23:40.929229
##################################################
1054.csv
**************************************************
2022-02-24 23:24:07.297680
2022-02-24 23:24:13.876080
2022-02-24 23:24:47.102183
##################################################
1055.csv
**************************************************
2022-02-24 23:25:08.930781
2022-02-24 23:25:15.640829
2022-02-24 23:25:48.725339
########################################

In [13]:
file_path = 'E:\datatest\PseDPC\d10\d106'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
1061.csv
**************************************************
2022-02-24 23:30:19.222619
2022-02-24 23:30:25.518781
2022-02-24 23:30:56.589622
##################################################
1062.csv
**************************************************
2022-02-24 23:31:21.864997
2022-02-24 23:31:28.406496
2022-02-24 23:32:01.069129
##################################################
1063.csv
**************************************************
2022-02-24 23:32:29.392329
2022-02-24 23:32:35.637641
2022-02-24 23:33:06.525999
##################################################
1064.csv
**************************************************
2022-02-24 23:33:35.076591
2022-02-24 23:33:41.758741
2022-02-24 23:34:15.684944
##################################################
1065.csv
**************************************************
2022-02-24 23:34:49.466204
2022-02-24 23:34:56.181260
2022-02-24 23:35:29.388420
########################################

In [14]:
file_path = 'E:\datatest\PseDPC\d10\d107'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
1071.csv
**************************************************
2022-02-24 23:39:58.799584
2022-02-24 23:40:04.856379
2022-02-24 23:40:34.000406
##################################################
1072.csv
**************************************************
2022-02-24 23:41:00.936338
2022-02-24 23:41:07.163705
2022-02-24 23:41:38.411102
##################################################
1073.csv
**************************************************
2022-02-24 23:42:05.281183
2022-02-24 23:42:11.541434
2022-02-24 23:42:42.721041
##################################################
1074.csv
**************************************************
2022-02-24 23:43:10.399959
2022-02-24 23:43:16.501633
2022-02-24 23:43:47.632365
##################################################
1075.csv
**************************************************
2022-02-24 23:44:15.628440
2022-02-24 23:44:21.764024
2022-02-24 23:44:52.905706
########################################

In [15]:
file_path = 'E:\datatest\PseDPC\d10\d108'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
1081.csv
**************************************************
2022-02-24 23:49:53.087570
2022-02-24 23:49:59.138409
2022-02-24 23:50:28.954635
##################################################
1082.csv
**************************************************
2022-02-24 23:50:56.454033
2022-02-24 23:51:02.384167
2022-02-24 23:51:31.887232
##################################################
1083.csv
**************************************************
2022-02-24 23:51:57.495716
2022-02-24 23:52:03.446795
2022-02-24 23:52:13.585689
##################################################
1084.csv
**************************************************
2022-02-24 23:52:24.360839
2022-02-24 23:52:30.395692
2022-02-24 23:53:00.084261
##################################################
1085.csv
**************************************************
2022-02-24 23:53:25.879247
2022-02-24 23:53:32.563385
2022-02-24 23:54:06.015863
########################################

In [16]:
file_path = 'E:\datatest\PseDPC\d10\d109'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
1091.csv
**************************************************
2022-02-24 23:58:42.707574
2022-02-24 23:58:49.113436
2022-02-24 23:59:19.033385
##################################################
1092.csv
**************************************************
2022-02-24 23:59:39.951419
2022-02-24 23:59:46.710363
2022-02-25 00:00:20.108003
##################################################
1093.csv
**************************************************
2022-02-25 00:00:42.073212
2022-02-25 00:00:49.045559
2022-02-25 00:01:24.114731
##################################################
1094.csv
**************************************************
2022-02-25 00:01:47.610867
2022-02-25 00:01:54.396711
2022-02-25 00:02:27.922015
##################################################
1095.csv
**************************************************
2022-02-25 00:02:50.000943
2022-02-25 00:02:56.667136
2022-02-25 00:03:30.408860
########################################

In [17]:
file_path = 'E:\datatest\PseDPC\d10\d1010'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
10101.csv
**************************************************
2022-02-25 00:07:52.243295
2022-02-25 00:07:58.675087
2022-02-25 00:08:30.980675
##################################################
10102.csv
**************************************************
2022-02-25 00:08:57.345115
2022-02-25 00:09:03.767931
2022-02-25 00:09:13.673429
##################################################
10103.csv
**************************************************
2022-02-25 00:09:21.264145
2022-02-25 00:09:27.952226
2022-02-25 00:10:02.261432
##################################################
10104.csv
**************************************************
2022-02-25 00:10:28.976954
2022-02-25 00:10:35.788731
2022-02-25 00:10:46.267715
##################################################
10105.csv
**************************************************
2022-02-25 00:10:54.048875
2022-02-25 00:11:00.812778
2022-02-25 00:11:11.283791
###################################

In [18]:
file_path = 'E:\datatest\PseDPC\d10\d1011'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
10111.csv
**************************************************
2022-02-25 00:14:19.760495
2022-02-25 00:14:26.035705
2022-02-25 00:14:57.496565
##################################################
10112.csv
**************************************************
2022-02-25 00:15:19.796891
2022-02-25 00:15:26.531876
2022-02-25 00:16:00.394283
##################################################
10113.csv
**************************************************
2022-02-25 00:16:22.695611
2022-02-25 00:16:29.269993
2022-02-25 00:17:02.294636
##################################################
10114.csv
**************************************************
2022-02-25 00:17:24.583025
2022-02-25 00:17:31.176362
2022-02-25 00:18:04.410446
##################################################
10115.csv
**************************************************
2022-02-25 00:18:32.674824
2022-02-25 00:18:39.417783
2022-02-25 00:19:11.349350
###################################

In [19]:
file_path = 'E:\datatest\PseDPC\d10\d1012'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
10121.csv
**************************************************
2022-02-25 00:23:40.181093
2022-02-25 00:23:46.938036
2022-02-25 00:23:57.161689
##################################################
10122.csv
**************************************************
2022-02-25 00:24:07.781249
2022-02-25 00:24:14.648874
2022-02-25 00:24:48.825470
##################################################
10123.csv
**************************************************
2022-02-25 00:25:15.492110
2022-02-25 00:25:22.755683
2022-02-25 00:25:57.901622
##################################################
10124.csv
**************************************************
2022-02-25 00:26:20.084294
2022-02-25 00:26:27.199236
2022-02-25 00:26:37.657256
##################################################
10125.csv
**************************************************
2022-02-25 00:26:45.162209
2022-02-25 00:26:51.987942
2022-02-25 00:27:02.502811
###################################

In [20]:
file_path = 'E:\datatest\PseDPC\d10\d1013'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
10131.csv
**************************************************
2022-02-25 00:30:14.637726
2022-02-25 00:30:21.586164
2022-02-25 00:30:56.582524
##################################################
10132.csv
**************************************************
2022-02-25 00:31:27.086889
2022-02-25 00:31:34.376413
2022-02-25 00:32:10.732144
##################################################
10133.csv
**************************************************
2022-02-25 00:32:33.020483
2022-02-25 00:32:40.075607
2022-02-25 00:33:15.275431
##################################################
10134.csv
**************************************************
2022-02-25 00:33:41.625930
2022-02-25 00:33:48.672078
2022-02-25 00:34:23.804176
##################################################
10135.csv
**************************************************
2022-02-25 00:34:50.173532
2022-02-25 00:34:57.237632
2022-02-25 00:35:07.451338
###################################

In [21]:
file_path = 'E:\datatest\PseDPC\d10\d1014'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
10141.csv
**************************************************
2022-02-25 00:37:49.295293
2022-02-25 00:37:56.412251
2022-02-25 00:38:31.460501
##################################################
10142.csv
**************************************************
2022-02-25 00:38:53.485579
2022-02-25 00:39:00.255439
2022-02-25 00:39:33.330967
##################################################
10143.csv
**************************************************
2022-02-25 00:39:56.000294
2022-02-25 00:40:03.561066
2022-02-25 00:40:40.091328
##################################################
10144.csv
**************************************************
2022-02-25 00:41:02.354763
2022-02-25 00:41:09.729060
2022-02-25 00:41:45.074496
##################################################
10145.csv
**************************************************
2022-02-25 00:42:07.271101
2022-02-25 00:42:14.486775
2022-02-25 00:42:49.532011
###################################

In [22]:
file_path = 'E:\datatest\PseDPC\d10\d1015'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
10151.csv
**************************************************
2022-02-25 00:47:41.582631
2022-02-25 00:47:48.693633
2022-02-25 00:48:23.324950
##################################################
10152.csv
**************************************************
2022-02-25 00:48:45.409957
2022-02-25 00:48:52.024198
2022-02-25 00:49:24.733651
##################################################
10153.csv
**************************************************
2022-02-25 00:49:53.003038
2022-02-25 00:49:59.996333
2022-02-25 00:50:35.768598
##################################################
10154.csv
**************************************************
2022-02-25 00:50:58.242496
2022-02-25 00:51:05.171929
2022-02-25 00:51:41.143688
##################################################
10155.csv
**************************************************
2022-02-25 00:52:03.674439
2022-02-25 00:52:10.533083
2022-02-25 00:52:20.505374
###################################

In [23]:
file_path = 'E:\datatest\PseDPC\d10\d1016'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
10161.csv
**************************************************
2022-02-25 00:55:23.497801
2022-02-25 00:55:29.674254
2022-02-25 00:56:01.065268
##################################################
10162.csv
**************************************************
2022-02-25 00:56:28.160809
2022-02-25 00:56:35.236872
2022-02-25 00:57:09.487234
##################################################
10163.csv
**************************************************
2022-02-25 00:57:35.811774
2022-02-25 00:57:42.653476
2022-02-25 00:58:16.796121
##################################################
10164.csv
**************************************************
2022-02-25 00:58:43.044892
2022-02-25 00:58:49.936454
2022-02-25 00:59:24.195821
##################################################
10165.csv
**************************************************
2022-02-25 00:59:52.590823
2022-02-25 00:59:58.767328
2022-02-25 01:00:29.561940
###################################

In [24]:
file_path = 'E:\datatest\PseDPC\d10\d1017'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
10171.csv
**************************************************
2022-02-25 01:04:44.469901
2022-02-25 01:04:50.955577
2022-02-25 01:05:23.419725
##################################################
10172.csv
**************************************************
2022-02-25 01:05:49.558784
2022-02-25 01:05:56.611914
2022-02-25 01:06:31.037780
##################################################
10173.csv
**************************************************
2022-02-25 01:07:01.872320
2022-02-25 01:07:08.867567
2022-02-25 01:07:43.452065
##################################################
10174.csv
**************************************************
2022-02-25 01:08:13.971415
2022-02-25 01:08:20.929765
2022-02-25 01:08:55.336739
##################################################
10175.csv
**************************************************
2022-02-25 01:09:25.703491
2022-02-25 01:09:32.729692
2022-02-25 01:10:06.990002
###################################

In [25]:
file_path = 'E:\datatest\PseDPC\d10\d1018'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
10181.csv
**************************************************
2022-02-25 01:15:18.175457
2022-02-25 01:15:24.695980
2022-02-25 01:15:57.687711
##################################################
10182.csv
**************************************************
2022-02-25 01:16:24.178835
2022-02-25 01:16:31.104340
2022-02-25 01:17:05.722713
##################################################
10183.csv
**************************************************
2022-02-25 01:17:32.039275
2022-02-25 01:17:38.863051
2022-02-25 01:18:13.319856
##################################################
10184.csv
**************************************************
2022-02-25 01:18:39.610515
2022-02-25 01:18:46.470135
2022-02-25 01:19:20.847159
##################################################
10185.csv
**************************************************
2022-02-25 01:19:49.102563
2022-02-25 01:19:55.883442
2022-02-25 01:20:30.146749
###################################

In [26]:
file_path = 'E:\datatest\PseDPC\d10\d1019'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
10191.csv
**************************************************
2022-02-25 01:25:18.596038
2022-02-25 01:25:24.965989
2022-02-25 01:25:56.011927
##################################################
10192.csv
**************************************************
2022-02-25 01:26:22.304586
2022-02-25 01:26:28.646614
2022-02-25 01:26:59.768346
##################################################
10193.csv
**************************************************
2022-02-25 01:27:29.362169
2022-02-25 01:27:35.762045
2022-02-25 01:28:07.018424
##################################################
10194.csv
**************************************************
2022-02-25 01:28:33.008881
2022-02-25 01:28:39.351904
2022-02-25 01:29:10.534482
##################################################
10195.csv
**************************************************
2022-02-25 01:29:36.504998
2022-02-25 01:29:43.876248
2022-02-25 01:30:19.074110
###################################

In [27]:
file_path = 'E:\datatest\PseDPC\d10\d1020'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
10201.csv
**************************************************
2022-02-25 01:35:18.324462
2022-02-25 01:35:24.197754
2022-02-25 01:35:33.701294
##################################################
10202.csv
**************************************************
2022-02-25 01:35:40.961896
2022-02-25 01:35:47.189234
2022-02-25 01:36:18.747802
##################################################
10203.csv
**************************************************
2022-02-25 01:36:44.386204
2022-02-25 01:36:50.546721
2022-02-25 01:37:22.151137
##################################################
10204.csv
**************************************************
2022-02-25 01:37:47.988010
2022-02-25 01:37:54.319100
2022-02-25 01:38:25.707121
##################################################
10205.csv
**************************************************
2022-02-25 01:38:53.595506
2022-02-25 01:38:59.804892
2022-02-25 01:39:33.310222
###################################